<a href="https://colab.research.google.com/github/feiyoung/ReadPapers/blob/master/daegc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DAEGC by pytorch

A PyTorch implement of [Attributed Graph Clustering: A Deep Attentional Embedding Approach](https://www.ijcai.org/Proceedings/2019/0509.pdf)

## Result On Cora

|Model|ACC|NMI|F-score|ARI|
|---|---|---|---|---|
|DAEGC|0.704|0.528|0.682|0.496|
|my DAEGC|0.707|0.543|0.693|0.484|

*Didn't run kmeans 50 times to get average score for my implement*

*Just take the best score*

## Result On Citeseer

|Model|ACC|NMI|F-score|ARI|
|---|---|---|---|---|
|DAEGC|0.672|0.397|0.636|0.410|
|my DAEGC|0.687|0.417|0.643|0.441|

*Didn't run kmeans 50 times to get average score for my implement*

*Just take the best score*

## Prepare

### Check GPU and CUDA version

In [ ]:
!nvidia-smi

Mon Mar 22 01:45:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


### Install requirement

In [ ]:
%%bash
#CUDA=cpu
CUDA=cu110
TORCH=1.7.0
TORCHVISION=0.8.1

echo BUILDING WITH CUDA===${CUDA} AND TORCH===${TORCH}
python3 -m pip install torch==${TORCH}+${CUDA} torchvision==${TORCHVISION}+${CUDA} -f https://download.pytorch.org/whl/torch_stable.html

python3 -m pip install torch-scatter==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-${TORCH}.html
python3 -m pip install torch-sparse==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-${TORCH}.html
python3 -m pip install torch-cluster==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-${TORCH}.html
python3 -m pip install torch-spline-conv==latest+${CUDA} -f https://pytorch-geometric.com/whl/torch-${TORCH}.html
python3 -m pip install torch-geometric

BUILDING WITH CUDA===cu110 AND TORCH===1.7.0
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchvision 0.9.0+cu101
    Uninstalling torchvision-0.9.0+cu101:
      Successfully uninstalled torchvision-0.9.0+cu101
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
  Created wheel for torch-geometric: filename=torch_geometric-1.6.3-cp37-none-any.whl size=322719 sha256=d80dc981d5ab73d2cf0e9575a156c846a6bc9eb87f2b741cd08e2d9e925fccb9
  Stored in directory: /root/.cache/pip/wheels/6d/47/1e/0af8ce3e21783c3e584c22502011a3367c091694eebc50a971
Successfully built torch-geometric


tcmalloc: large alloc 1147494400 bytes == 0x55efa165c000 @  0x7f234813d615 0x55ef673fc06c 0x55ef674dbeba 0x55ef673fee8d 0x55ef674f099d 0x55ef67472fe9 0x55ef6746db0e 0x55ef6740077a 0x55ef67472e50 0x55ef6746db0e 0x55ef6740077a 0x55ef6746f86a 0x55ef674f17c6 0x55ef6746eee2 0x55ef674f17c6 0x55ef6746eee2 0x55ef674f17c6 0x55ef6746eee2 0x55ef674f17c6 0x55ef67573431 0x55ef674d4049 0x55ef6743ec84 0x55ef673ff8e9 0x55ef67473ade 0x55ef6740069a 0x55ef6746ea45 0x55ef6746de0d 0x55ef6740077a 0x55ef6746ea45 0x55ef6740069a 0x55ef6746ea45
tcmalloc: large alloc 1434370048 bytes == 0x55efe5cb2000 @  0x7f234813d615 0x55ef673fc06c 0x55ef674dbeba 0x55ef673fee8d 0x55ef674f099d 0x55ef67472fe9 0x55ef6746db0e 0x55ef6740077a 0x55ef67472e50 0x55ef6746db0e 0x55ef6740077a 0x55ef6746f86a 0x55ef674f17c6 0x55ef6746eee2 0x55ef674f17c6 0x55ef6746eee2 0x55ef674f17c6 0x55ef6746eee2 0x55ef674f17c6 0x55ef67573431 0x55ef674d4049 0x55ef6743ec84 0x55ef673ff8e9 0x55ef67473ade 0x55ef6740069a 0x55ef6746ea45 0x55ef6746de0d 0x55ef6740

In [ ]:
%%bash
pip3 install munkres
pip3 install -U scikit-learn

  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


### Download code from github

In [ ]:
%%bash
git clone https://github.com/Tiger101010/DAEGC.git
mkdir /content/DAEGC/DAEGC/pretrain
rm -rf /content/sample_data/

Cloning into 'DAEGC'...


## Pretrain

### Cora

In [ ]:
%%bash
cd /content/DAEGC/DAEGC
python3 pretrain.py --name Cora --max_epoch 50

use cuda: True
Namespace(alpha=0.2, cuda=True, embedding_size=16, hidden_size=256, input_dim=1433, k=None, lr=0.005, max_epoch=50, n_clusters=7, name='Cora', weight_decay=0.005)
GAT(
  (conv1): GATLayer (1433 -> 256)
  (conv2): GATLayer (256 -> 16)
)
epoch 0:acc 0.4431, nmi 0.2114, ari 0.1504, f1 0.4337
epoch 1:acc 0.4553, nmi 0.2854, ari 0.2044, f1 0.4499
epoch 2:acc 0.4982, nmi 0.3375, ari 0.2525, f1 0.4949
epoch 3:acc 0.5063, nmi 0.3592, ari 0.2763, f1 0.5034
epoch 4:acc 0.5225, nmi 0.3812, ari 0.2914, f1 0.5047
epoch 5:acc 0.5727, nmi 0.4272, ari 0.3386, f1 0.5810
epoch 6:acc 0.5805, nmi 0.4316, ari 0.3455, f1 0.5790
epoch 7:acc 0.5857, nmi 0.4365, ari 0.3465, f1 0.5827
epoch 8:acc 0.5934, nmi 0.4476, ari 0.3575, f1 0.5894
epoch 9:acc 0.6049, nmi 0.4527, ari 0.3631, f1 0.6019
epoch 10:acc 0.6204, nmi 0.4634, ari 0.3789, f1 0.6160
epoch 11:acc 0.6185, nmi 0.4651, ari 0.3768, f1 0.6153
epoch 12:acc 0.6171, nmi 0.4658, ari 0.3763, f1 0.6135
epoch 13:acc 0.6193, nmi 0.4680, ari 0.3779,

### Citeseer


In [ ]:
%%bash
cd /content/DAEGC/DAEGC
python3 pretrain.py --name Citeseer --max_epoch 50

use cuda: True
Namespace(alpha=0.2, cuda=True, embedding_size=16, hidden_size=256, input_dim=3703, k=None, lr=0.005, max_epoch=50, n_clusters=6, name='Citeseer', weight_decay=0.005)
GAT(
  (conv1): GATLayer (3703 -> 256)
  (conv2): GATLayer (256 -> 16)
)
epoch 0:acc 0.3303, nmi 0.1065, ari 0.0883, f1 0.3097
epoch 1:acc 0.3883, nmi 0.1517, ari 0.1280, f1 0.3711
epoch 2:acc 0.4788, nmi 0.2179, ari 0.1969, f1 0.4601
epoch 3:acc 0.5029, nmi 0.2466, ari 0.2240, f1 0.4838
epoch 4:acc 0.5242, nmi 0.2689, ari 0.2467, f1 0.5002
epoch 5:acc 0.5437, nmi 0.2957, ari 0.2715, f1 0.5168
epoch 6:acc 0.5573, nmi 0.3102, ari 0.2865, f1 0.5282
epoch 7:acc 0.5654, nmi 0.3201, ari 0.2964, f1 0.5354
epoch 8:acc 0.5693, nmi 0.3324, ari 0.3059, f1 0.5371
epoch 9:acc 0.5756, nmi 0.3407, ari 0.3148, f1 0.5446
epoch 10:acc 0.5753, nmi 0.3449, ari 0.3151, f1 0.5453
epoch 11:acc 0.5852, nmi 0.3549, ari 0.3274, f1 0.5547
epoch 12:acc 0.5840, nmi 0.3551, ari 0.3283, f1 0.5542
epoch 13:acc 0.5858, nmi 0.3529, ari 0.3

### Pubmed

In [ ]:
%%bash
cd /content/DAEGC/DAEGC
python3 pretrain.py --name PubMed --max_epoch 1

## Train

### Cora

|Model|ACC|NMI|F-score|ARI|
|---|---|---|---|---|
|DAEGC|0.704|0.528|0.682|0.496|
|my DAEGC|0.707|0.543|0.693|0.484|

In [ ]:
%%bash
cd /content/DAEGC/DAEGC
python3 daegc.py --update_interval 5 --name Cora --epoch 29 --max_epoch 200

use cuda: True
Namespace(alpha=0.2, cuda=True, embedding_size=16, epoch=29, hidden_size=256, input_dim=1433, k=None, lr=0.0001, max_epoch=200, n_clusters=7, name='Cora', pretrain_path='./pretrain/predaegc_Cora_29.pkl', update_interval=5, weight_decay=0.005)
DAEGC(
  (gat): GAT(
    (conv1): GATLayer (1433 -> 256)
    (conv2): GATLayer (256 -> 16)
  )
)
epoch pretrain:acc 0.7068, nmi 0.5237, ari 0.4746, f1 0.6907
epoch 0:acc 0.7068, nmi 0.5237, ari 0.4746, f1 0.6907
epoch 5:acc 0.7075, nmi 0.5269, ari 0.4756, f1 0.6929
epoch 10:acc 0.7086, nmi 0.5307, ari 0.4784, f1 0.6940
epoch 15:acc 0.7075, nmi 0.5323, ari 0.4779, f1 0.6931
epoch 20:acc 0.7068, nmi 0.5316, ari 0.4786, f1 0.6914
epoch 25:acc 0.7057, nmi 0.5319, ari 0.4786, f1 0.6901
epoch 30:acc 0.7083, nmi 0.5362, ari 0.4834, f1 0.6928
epoch 35:acc 0.7072, nmi 0.5357, ari 0.4822, f1 0.6919
epoch 40:acc 0.7064, nmi 0.5357, ari 0.4812, f1 0.6914
epoch 45:acc 0.7064, nmi 0.5381, ari 0.4817, f1 0.6919
epoch 50:acc 0.7075, nmi 0.5393, ari

### Citeseer

|Model|ACC|NMI|F-score|ARI|
|---|---|---|---|---|
|DAEGC|0.672|0.397|0.636|0.410|
|my DAEGC|0.687|0.417|0.643|0.441|

In [ ]:
%%bash
cd /content/DAEGC/DAEGC
python3 daegc.py --update_interval 5 --name Citeseer --epoch 41 --max_epoch 200

use cuda: True
Namespace(alpha=0.2, cuda=True, embedding_size=16, epoch=41, hidden_size=256, input_dim=3703, k=None, lr=0.0001, max_epoch=200, n_clusters=6, name='Citeseer', pretrain_path='./pretrain/predaegc_Citeseer_41.pkl', update_interval=5, weight_decay=0.005)
DAEGC(
  (gat): GAT(
    (conv1): GATLayer (3703 -> 256)
    (conv2): GATLayer (256 -> 16)
  )
)
epoch pretrain:acc 0.6748, nmi 0.4005, ari 0.4207, f1 0.6295
epoch 0:acc 0.6748, nmi 0.4005, ari 0.4207, f1 0.6295
epoch 5:acc 0.6775, nmi 0.4062, ari 0.4259, f1 0.6318
epoch 10:acc 0.6772, nmi 0.4048, ari 0.4259, f1 0.6314
epoch 15:acc 0.6814, nmi 0.4098, ari 0.4311, f1 0.6374
epoch 20:acc 0.6826, nmi 0.4118, ari 0.4335, f1 0.6384
epoch 25:acc 0.6832, nmi 0.4116, ari 0.4341, f1 0.6393
epoch 30:acc 0.6850, nmi 0.4138, ari 0.4368, f1 0.6413
epoch 35:acc 0.6868, nmi 0.4160, ari 0.4399, f1 0.6425
epoch 40:acc 0.6874, nmi 0.4174, ari 0.4412, f1 0.6430
epoch 45:acc 0.6895, nmi 0.4197, ari 0.4442, f1 0.6453
epoch 50:acc 0.6871, nmi 0.4

### Pubmed

In [ ]:
%%bash
cd /content/DAEGC/DAEGC
python3 daegc.py --update_interval 5 --name PubMed --epoch 25 --max_epoch 200